# **Notebook D.** Classification Using a Transformer Model
----

One of the biggest recent developments in Natural Language Processing has come from the introduction of Transformer models (e.g. *BERT, EARNIE, RoBERTa*, etc.). The idea is that a model is trained on a very large corpus and used to create an embedding represention of words. This "raw" model can be downloaded and then fine-tuned (retrained) on our own data.

There are several ways to implement these models. Researchers who are most comfortable with Python may start with the **Transformers** library by **HuggingFace** (https://huggingface.co/transformers/). This is the most flexible approach, but it also requires effort for researchers to implement.

An alterantive is the **SimpleTransformers** library which is a wrapper for this functionality. This library contains an easy-to-use version of this transformer technique (https://simpletransformers.ai) that is similar to the sklearn commands we have used thus far.

This package is not pre-installed with colab. To do this, we need perform the following:
 - 1) run *!pip install simpletransformers* in the notebook below
 - 2) Comment out the code by putting a # in front of the line (e.g. *#!pip install simpletransformers*)
 - 3) Rerun all of the code from the top menu (or hit Ctrl+F9)




Models: https://simpletransformers.ai/docs/classification-specifics/#supported-model-types

Model Options: https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model


In [9]:
!pip install Transformers==4.44.1 --upgrade

In [10]:
!pip install simpletransformers

In [11]:
# Load the Simple Transformers Package for Text Classification

from simpletransformers.classification import ClassificationModel

In [12]:
# Turn of warnings, just to avoid pesky messages that might cause confusion here
# Remove when testing your own code #
import warnings
warnings.filterwarnings("ignore")

In [13]:
import logging

logging.basicConfig(level=logging.ERROR)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

# D.1. Preamble: Load Packages
---

In [14]:
# General Packages #
import os
import pandas as pd
import numpy as np

# TQDM to Show Progress Bars #
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

# SKLearn libraries for splitting sample and validation
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Additional Libraries that we are using only in this notebook
import torch
import gc

In [15]:
# Turn of warnings, just to avoid pesky messages that might cause confusion here
# Remove when testing your own code #
import warnings
warnings.filterwarnings("ignore")

In [16]:
# Mount Personal Google Drive on own Machine -- You have to follow the link to log in #
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# D.2. Load Training Data ##
----------------

We are going to use the data on the Google drive. This is in a csv file, and so we are going to load the data as a dataframe, and then convert the main data (Patent Ids, Indicator for AI / Non-AI, Patent Abstract) from a Pandas DataFrame to a list (which is more easily used in later sections).

In [17]:
# Change to Working Directory with Training Data #
os.chdir("/content/drive/MyDrive/USPTO_data")

# Load Training Data #
TrainingData = pd.read_csv("./Training_Data/4K Patents - AI 20p.csv")

# Store Data in Lists for Text Classification #
IDs = np.array(TrainingData['app number'].values.tolist())
Abstract_Text = TrainingData['abstract'].values.tolist()
Classes = TrainingData['actual'].values.tolist()

# D.3. Perform Classification with Transformer Model
---

As before, we are going to go through different models and compare their performance. Recall that transformer models are pre-trained by an external entity and we are simply downloading them (pre-trained) from the web and fine tuning them our particular application.

We download these models from hugging face. We are using the simpletransformers library which allows us to automatically download and train these models, using the same basic command for different models. We simply need to specify the model architecture (e.g. BERT) and then specific model, which usually refers to the type of data it was trained on (e.g. bert-base-uncased).

In the following link you can see the possible models that can be used by simple-transformers.

* https://simpletransformers.ai/docs/classification-specifics/#supported-model-types

This refers to the model type or architecture. There might be various types of models trained for different purposes that use the same architecture (e.g. SciBERT). You can downlaod the most common models directly from huggingface:

* https://huggingface.co/transformers/pretrained_models.html

You can also download community models here:

* https://huggingface.co/models

Below we define a list of the different transformer models we are going to use. These are listed in the following order: Name (e.g. BERT), Architecture (e.g. bert), Specific Model (e.g. bert-base-uncased)


In [18]:
# ===== Stable, no-freeze rewrite =====
import os, gc, torch, numpy as np, pandas as pd
from tqdm.auto import tqdm
os.environ["TOKENIZERS_PARALLELISM"] = "false"   # 避免分词并行卡住

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

In [23]:
# ✅ 严格按 simpletransformers 支持的 model_type（classification-specifics）
CLASSIFIERS = [
    ["BERT",        "bert",        "bert-base-uncased"],
    ["RoBERTa",     "roberta",     "roberta-base"],
    ["DeBERTa",     "deberta",     "microsoft/deberta-base"],          # v1 架构，对应 model_type='deberta'
    # ["Longformer",  "longformer",  "allenai/longformer-base-4096"],    # 长序列
    ["BigBird",     "bigbird",     "google/bigbird-roberta-base"],     # 长序列
    ["DistilBERT",  "distilbert",  "distilbert-base-uncased"],
    ["ALBERT",      "albert",      "albert-base-v2"],
    ["SciBERT",     "bert",        "allenai/scibert_scivocab_uncased"],# 属于 BERT 架构
    ["PatentBERT",  "bert",        "anferico/bert-for-patents"],       # 属于 BERT 架构
    ["BioBERT",     "bert",        "dmis-lab/biobert-v1.1"],           # 属于 BERT 架构
    ["XLNet",       "xlnet",       "xlnet-base-cased"],
    ["ELECTRA",     "electra",     "google/electra-base-discriminator"],
]


In [24]:
# ===== 配置 =====
NUM_OF_SPLITS = 2
Reweight = True

RESULTS, Classified_Values = [], []

BASE_ARGS = {
    "num_train_epochs": 1,
    "train_batch_size": 32,
    "eval_batch_size": 64,
    "max_seq_length": 512,          # 统一 512（已去掉 Longformer/4096/滑窗）
    "fp16": True,
    "overwrite_output_dir": True,
    "use_early_stopping": True,
    "early_stopping_patience": 1,
    "early_stopping_metric": "eval_loss",
    "reprocess_input_data": True,
    "no_save": True,
    "no_cache": True,
    "silent": False,
    "logging_steps": 20,
    "process_count": 1,
    "use_multiprocessing": False,
    "use_multiprocessing_for_evaluation": False,
    "dataloader_num_workers": 0,
}

def build_args_for(model_type, model_name):
    # 仅处理 uncased 的小写化，其它保持默认
    args = BASE_ARGS.copy()
    args["do_lower_case"] = ("uncased" in model_name.lower())
    return args

def logits_to_prob(raw_outputs):
    raw = np.array(raw_outputs)
    raw = np.squeeze(raw)  # 兼容 (N,1,2) 等形状
    if raw.ndim == 2 and raw.shape[1] == 2:
        raw = raw - raw.max(axis=1, keepdims=True)
        exp = np.exp(raw); probs = exp / exp.sum(axis=1, keepdims=True)
        return probs[:, 1]
    elif raw.ndim == 1:
        z = raw.squeeze()
        return 1.0 / (1.0 + np.exp(-z))
    else:
        raise ValueError(f"Unexpected raw_outputs shape: {raw.shape}")

# ——把标签统一成 1 维 0/1（一次到位）——
def normalize_binary_labels(arr):
    a = np.asarray(arr)
    if a.ndim == 1 and a.dtype == object:
        try: a = np.stack(a)
        except Exception: pass
    if a.ndim == 2 and a.shape[1] == 2:
        if np.array_equal(a, a.astype(int)):   # one-hot
            a = a.argmax(axis=1).astype(int)
        else:                                   # 概率 [p0, p1]
            a = (a[:, 1] >= 0.5).astype(int)
    if a.ndim != 1:
        raise ValueError(f"标签形状仍不是1维，当前 {a.shape}")
    return a.astype(int)

LABELS_1D = normalize_binary_labels(Classes)

# ===== 主循环：每个模型 × K 折 =====
use_cuda = torch.cuda.is_available()
for name, model_type, model_name in tqdm(CLASSIFIERS, desc="Evaluating Classifiers", leave=True):
    y_actual, y_predicted, id_s = [], [], []
    prob_pos_all = []

    kf = StratifiedKFold(n_splits=NUM_OF_SPLITS, shuffle=True, random_state=1)
    for train_i, test_i in tqdm(kf.split(Abstract_Text, LABELS_1D),
                                desc=f"{name} | Cross-Validating",
                                leave=False, total=NUM_OF_SPLITS):

        X = np.array(Abstract_Text); Y = LABELS_1D
        train_X, test_X = X[train_i], X[test_i]
        train_y, test_y = Y[train_i], Y[test_i]
        Train_IDs, Test_IDs = IDs[train_i], IDs[test_i]

        train_df = pd.DataFrame({"text": list(train_X), "labels": list(train_y)})

        # 类别权重：支持则传，不支持（如 DeBERTa）则自动去掉重试
        weight_vec = None
        if Reweight:
            from sklearn.utils.class_weight import compute_class_weight
            cls_w = compute_class_weight(class_weight="balanced", classes=np.array([0,1]), y=train_y)
            weight_vec = cls_w.tolist()

        args = build_args_for(model_type, model_name)

        try:
            model = ClassificationModel(
                model_type, model_name,
                weight=weight_vec,
                args=args,
                use_cuda=use_cuda
            )
        except ValueError as e:
            if "does not currently support class weights" in str(e).lower():
                model = ClassificationModel(
                    model_type, model_name,
                    weight=None,
                    args=args,
                    use_cuda=use_cuda
                )
            else:
                raise

        # 训练
        model.train_model(train_df)

        # 预测：拿 logits → 概率（算 AUC）
        preds, raw_outputs = model.predict(list(test_X))
        prob_pos = logits_to_prob(raw_outputs)

        # 累积
        id_s.extend(list(Test_IDs))
        y_actual.extend(list(test_y))
        y_predicted.extend(list(preds))
        prob_pos_all.extend(list(prob_pos))

        gc.collect(); torch.cuda.empty_cache()

    # ===== 汇总指标（与原格式一致） =====
    y_arr = np.asarray(y_actual); p_arr = np.asarray(prob_pos_all)

    Share = np.round(np.mean(y_predicted), 3)
    Accuracy = accuracy_score(y_arr, y_predicted)
    ROC = roc_auc_score(y_arr, p_arr)
    Precision = precision_score(y_arr, y_predicted, zero_division=0)
    Recall = recall_score(y_arr, y_predicted, zero_division=0)
    F1 = f1_score(y_arr, y_predicted, zero_division=0)

    tn, fp, fn, tp = confusion_matrix(y_arr, y_predicted).ravel()
    TPR = round(tp/(tp+fn), 3)
    FNR = round(fn/(tp+fn), 3)
    FPR = round(fp/(fp+tn), 3)
    TNR = round(tn/(tn+fp), 3)

    RESULTS.append([name, Share, TPR, FNR, FPR, TNR,
                    round(Accuracy,3), round(ROC,3),
                    round(Precision,3), round(Recall,3), round(F1,3)])

    Classified_Values.append(list(zip(len(id_s)*[name], id_s, y_actual, y_predicted)))

Evaluating Classifiers:   0%|          | 0/11 [00:00<?, ?it/s]

BERT | Cross-Validating:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 1 of 1:   0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

# D.4. Output Classification Results #
----


In [25]:
# Convert List to Dataframe #
RESULTS_TABLE = pd.DataFrame(RESULTS, columns = ["Name", "Share", "True-Positives",
                                                 "False-Negatives", "False-Positives",
                                                 "True-Negatives","Accuracy", "AUC",
                                                 "Precision", "Recall", "F1"] )

RESULTS_TABLE["Type"] = "Transformer"
RESULTS_TABLE = RESULTS_TABLE[["Name", "Type", "Share", "True-Positives",
                               "False-Negatives", "False-Positives",
                               "True-Negatives","Accuracy", "AUC",
                               "Precision", "Recall", "F1"]]



# Output Results #
RESULTS_TABLE.sort_values("Accuracy", ascending = False ).to_csv("./Output/Model Performance/Transformer Classification Model Performance.csv")

# Display Results -- Out of Sample (Holdout) prediction -- Sorted by Accuracy #
RESULTS_TABLE.sort_values("Accuracy", ascending = False )


,Name,Type,Share,True-Positives,False-Negatives,False-Positives,True-Negatives,Accuracy,AUC,Precision,Recall,F1
2,DeBERTa,Transformer,0.214,0.869,0.131,0.050,0.950,0.934,0.971,0.814,0.869,0.841
8,BioBERT,Transformer,0.230,0.900,0.100,0.062,0.938,0.930,0.966,0.785,0.900,0.839
6,SciBERT,Transformer,0.244,0.919,0.081,0.075,0.925,0.923,0.972,0.754,0.919,0.828
4,DistilBERT,Transformer,0.235,0.890,0.110,0.070,0.930,0.922,0.967,0.760,0.890,0.820
1,RoBERTa,Transformer,0.238,0.900,0.100,0.072,0.928,0.922,0.970,0.758,0.900,0.823
7,PatentBERT,Transformer,0.248,0.918,0.082,0.080,0.920,0.920,0.970,0.743,0.918,0.821
10,ELECTRA,Transformer,0.248,0.885,0.115,0.088,0.912,0.907,0.954,0.717,0.885,0.792
0,BERT,Transformer,0.250,0.887,0.113,0.090,0.910,0.905,0.961,0.712,0.887,0.790
3,BigBird,Transformer,0.283,0.936,0.064,0.119,0.881,0.892,0.971,0.663,0.936,0.777
9,XLNet,Transformer,0.299,0.931,0.069,0.140,0.860,0.874,0.957,0.625,0.931,0.748


In [26]:
# Output Classification Results for Training Dataset -- PREDICTED VALUES -- Out Of Sample (Holdout) Prediction #

for i in range(0,len(Classified_Values), 1):

  Temp = pd.DataFrame(  Classified_Values[i],
                        columns = ['Model', 'id', 'Actual', 'Predicted'] )

  if i == 0:
    name = Temp.head(1)['Model'][0]
    Temp = Temp[['id', 'Actual', 'Predicted']]
    Temp.columns = ['id', 'Actual', name]
    Final = Temp

  else:

    name = Temp.head(1)['Model'][0]
    Temp = Temp[['id', 'Predicted']]
    Temp.columns = ['id', name]

    Final = Final.merge(Temp, on = ['id'])

# Save Data Frame #
Final.to_csv("./Output/Classification Output/Transformer Classification Results.csv")

Delete files that were created behind the scenes by the transformer model.

In [27]:
rm -rf "./outputs"

In [28]:
rm -rf "./runs"